In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
#MODEL = "mixtral:8x7b"
MODEL = "llama3.2"


In [5]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings



model = Ollama(model=MODEL)
#model = Ollama(model=MODEL, memory_optimized=True)

embeddings = OllamaEmbeddings(model=MODEL)

model.invoke("Tell me a joke")

"Why couldn't the bicycle stand up by itself?\n\nBecause it was two-tired."

In [6]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
chain.invoke("Tell me a joke")

"Here's one:\n\nWhat do you call a fake noodle?\n\nAn impasta!"

In [7]:
from langchain.prompts import PromptTemplate

template = """
You are an expert in cars. Use the following context to answer the question.

Context: {context}
Question: {question}

Answer:
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nYou are an expert in cars. Use the following context to answer the question.\n\nContext: Here is some context\nQuestion: Here is a question\n\nAnswer:\n'

In [8]:
chain = prompt | model | parser

chain.invoke({"context": "My car is red", "question": "What's your car'?"})

'My friend, my car is a sleek black sports car - a real head-turner! But I have to say, that new red car you mentioned? It looks fantastic! Is it a brand new purchase or an old reliable companion?'

In [9]:
from langchain_community.document_loaders import PyPDFLoader

pdf_files = ["book1.pdf", "book2.pdf", "book3.pdf", "book4.pdf", "book5.pdf"]  

pages = []

for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)  
    page = loader.load_and_split()  
    pages.extend(pages) 


In [10]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

C:\Users\LENOVO\Desktop\heba\RAG\diff concept\virt\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [11]:
retriever = vectorstore.as_retriever()
retriever.invoke("machine learning")

[]

In [12]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [13]:
questions = [
    "What is the fastest car?",
    "How many hours can a benz run?",
    "how to fix a water leak?",
    "how to fix a window crack?",
    "How much does a ford car cost?",
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: What is the fastest car?
Answer: A question that gets to the heart of many car enthusiasts!

The fastest production car in the world is the Bugatti Chiron Super Sport 300+.

With a top speed of 330 miles per hour (531 kilometers per hour), this hypercar is not only the fastest production car on the planet but also one of the most exclusive and sought-after vehicles in the world.

The Chiron Super Sport 300+ boasts an 8.0-liter, 16-cylinder engine that produces a staggering 1,600 horsepower, making it one of the most powerful production cars ever made.

It's worth noting that there are other cars that have achieved higher top speeds in testing or one-off runs, such as the Hennessey Venom F5 and the Koenigsegg Agera RS. However, these vehicles are not mass-produced and are often built for specific owners or racing applications.

So, if you're looking for the fastest production car that's widely available, the Bugatti Chiron Super Sport 300+ is the one to beat!

Question: How ma

In [14]:
def RAG(Q):
    
    answer= chain.invoke({'question': Q})
    return answer

In [15]:
import tkinter as tk
from tkinter import filedialog, ttk
import customtkinter as ctk

In [23]:
def answer():
    question = query_entry.get()
    question.strip
    response = RAG(question)
    result_text.delete(1.0, tk.END)
    result_text.insert(tk.END, response)
  

In [32]:
root = tk.Tk()
root.title("RAG")
root.geometry("800x600")
root.configure(bg="#ADD8E6")

In [33]:
title_label = tk.Label(root, text="I'm an expert in cars! Ask me and I'll help:", font=("Arial", 24, "bold"), bg="#ADD8E6")
title_label.pack(pady=(10, 5))

In [34]:
query_entry = tk.Entry(root, width=60, font=("Arial", 16))
query_entry.pack(pady=5)

In [35]:
upload_button = ctk.CTkButton(master=root, text="Answer", command=answer)
upload_button.pack(pady=5)

In [36]:
result_text = tk.Text(root, height=30, width=70, wrap=tk.WORD, font=("Arial", 12))
result_text.pack(pady=10)


In [37]:
root.mainloop()
